# Experiment for Dataset3

Author: Xiaolong Liu & Kedong Chen

Created: Nov 23, 2023

# In this notebook, we only show all the experiment settings and how we trainning the networks. Since some networks are complicated and the limit of our computation, all the results will be showed in Dataset3 result.ipynb.

# Setting up

## Import essential modules and dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
# Use this to calculate the execution time
import time
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_trainset = datasets.CIFAR10(root='./data/CIFAR10', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data/CIFAR10', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


## Define functions

In [3]:
def get_train_test_loader(trainset, testset, batch_size = 64, random_seed = 123):
    """
    inputs: 
        trainset: train data (from torchvision)
        testset: test data (from torchvision)
        batch_size: number of data points in a batch
        random_seed: random seed to make the results reproducible
    returns: 
        trainloader: iteratable object with batches of train data
        testloader: iteratable object with batches of test data
    """
    torch.manual_seed(random_seed)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              shuffle = True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size,
                                             shuffle = False, num_workers=8)
    return trainloader, testloader

In [4]:
def accuracy(net, dataloader):
    """
    inputs:
        net: trained neural network
        dataloader: trainloader or testloader
    outputs:
        accuracy: accuracy of the trained network for the given data
    """
    # torch.no_grad() suppresses the computation of the gradient and speeds things up
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct/total
    return accuracy

In [5]:
def train(net, trainloader, testloader, num_epochs = 10, step_size = 0.01, print_interval = 2000):
    criterion = nn.CrossEntropyLoss()
    loss_recorder = []
    print("Epoch |  Interval  |     Loss     |  train  ")
    for epoch in range(num_epochs):
        temp_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            #optimizer.zero_grad()
            net.zero_grad()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            train_acc = correct/total
            
            with torch.no_grad():
                for param in net.parameters():
                    param -= step_size * param.grad

            temp_loss += loss.item()
            if i % print_interval == print_interval-1:
                average_loss = temp_loss / print_interval
                print('%3d   |   %5d    |    %.5f   |  %.3f  ' 
                              % (epoch + 1, i + 1, average_loss, train_acc))
                loss_recorder.append(average_loss)
                temp_loss = 0.0
    return net, loss_recorder

# Experiment

In [6]:
batch_size_use = 10
random_seed_use = 1234
trainloader_10, testloader_10 = get_train_test_loader(cifar_trainset, cifar_testset, batch_size=batch_size_use,
                                                random_seed=random_seed_use)
batch_size_use = 100
random_seed_use = 1234
trainloader_100, testloader_100 = get_train_test_loader(cifar_trainset, cifar_testset, batch_size=batch_size_use,
                                                random_seed=random_seed_use)
batch_size_use = 1000
random_seed_use = 1234
trainloader_1000, testloader_1000 = get_train_test_loader(cifar_trainset, cifar_testset, batch_size=batch_size_use,
                                                random_seed=random_seed_use)

## Net1

In [7]:
class Net_1(nn.Module):
    """
    A Neural network with three fc layers
    """
    def __init__(self):
        super(Net_1, self).__init__()
        self.fc1 = nn.Linear(3072, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.loss = nn.CrossEntropyLoss() 

    def forward(self, z):
        """
        inputs:
            z: input for neural net (feature vector or a batch of features)
        outputs:
            output: given the current values of the neural net
               parameters, calculates the output of the
               neural net for the input of z
        """
        
        z = z.view(z.shape[0], -1)
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        output = self.fc3(z)

        return output

In [8]:
net1_1 = Net_1()
tic1_1 = time.time()
net1_1, loss1_1 = train(net1_1, trainloader_10, testloader_10, num_epochs = 20, step_size = 0.01, print_interval = 2500)
toc1_1 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |    2500    |    1.84204   |  0.200  
  1   |    5000    |    1.58868   |  0.200  
  2   |    2500    |    1.48163   |  0.200  
  2   |    5000    |    1.43941   |  0.600  
  3   |    2500    |    1.34898   |  0.400  
  3   |    5000    |    1.33870   |  0.500  
  4   |    2500    |    1.25557   |  0.500  
  4   |    5000    |    1.25928   |  0.600  
  5   |    2500    |    1.18790   |  0.700  
  5   |    5000    |    1.17858   |  0.800  
  6   |    2500    |    1.11741   |  0.700  
  6   |    5000    |    1.11694   |  0.600  
  7   |    2500    |    1.04348   |  0.700  
  7   |    5000    |    1.06848   |  0.600  
  8   |    2500    |    0.98665   |  0.800  
  8   |    5000    |    1.01201   |  0.200  
  9   |    2500    |    0.91355   |  0.700  
  9   |    5000    |    0.96457   |  0.700  
 10   |    2500    |    0.86282   |  0.900  
 10   |    5000    |    0.90374   |  0.700  
 11   |    2500    |    0.80734   |  0.900  
 11   |   

In [9]:
net1_2 = Net_1()
tic1_2 = time.time()
net1_2, loss1_2 = train(net1_2, trainloader_100, testloader_100, num_epochs = 20, step_size = 0.01, print_interval = 250)
toc1_2 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |     250    |    2.22070   |  0.210  
  1   |     500    |    2.02724   |  0.280  
  2   |     250    |    1.89755   |  0.310  
  2   |     500    |    1.82149   |  0.360  
  3   |     250    |    1.75579   |  0.440  
  3   |     500    |    1.71232   |  0.460  
  4   |     250    |    1.67016   |  0.500  
  4   |     500    |    1.64207   |  0.320  
  5   |     250    |    1.60854   |  0.430  
  5   |     500    |    1.59044   |  0.440  
  6   |     250    |    1.55956   |  0.350  
  6   |     500    |    1.54711   |  0.410  
  7   |     250    |    1.52188   |  0.430  
  7   |     500    |    1.50286   |  0.400  
  8   |     250    |    1.47764   |  0.530  
  8   |     500    |    1.47130   |  0.500  
  9   |     250    |    1.44203   |  0.490  
  9   |     500    |    1.43850   |  0.490  
 10   |     250    |    1.41479   |  0.530  
 10   |     500    |    1.39929   |  0.540  
 11   |     250    |    1.37941   |  0.420  
 11   |   

In [10]:
net1_3 = Net_1()
tic1_3 = time.time()
net1_3, loss1_3 = train(net1_3, trainloader_1000, testloader_1000, num_epochs = 20, step_size = 0.01, print_interval = 25)
toc1_3 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |      25    |    2.29329   |  0.176  
  1   |      50    |    2.27698   |  0.212  
  2   |      25    |    2.25957   |  0.211  
  2   |      50    |    2.24090   |  0.205  
  3   |      25    |    2.22002   |  0.228  
  3   |      50    |    2.19965   |  0.269  
  4   |      25    |    2.17633   |  0.280  
  4   |      50    |    2.15645   |  0.251  
  5   |      25    |    2.13432   |  0.281  
  5   |      50    |    2.11528   |  0.257  
  6   |      25    |    2.09748   |  0.291  
  6   |      50    |    2.07667   |  0.283  
  7   |      25    |    2.06234   |  0.273  
  7   |      50    |    2.04308   |  0.282  
  8   |      25    |    2.02984   |  0.287  
  8   |      50    |    2.01223   |  0.283  
  9   |      25    |    2.00166   |  0.303  
  9   |      50    |    1.98157   |  0.318  
 10   |      25    |    1.97025   |  0.315  
 10   |      50    |    1.95842   |  0.310  
 11   |      25    |    1.94929   |  0.317  
 11   |   

In [19]:
torch.save(net1_1, "net1_1")
torch.save(net1_2, "net1_2")
torch.save(net1_3, "net1_3")

In [28]:
import csv

# Your data
data = [
    loss1_1,
    loss1_2,
    loss1_3,
    [toc1_1-tic1_1, toc1_2-tic1_2, toc1_3-tic1_3]
]

# Open a file to write
with open('model1_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in data:
        writer.writerow(row)

## Net2: try convolution layers

In [11]:
class Net_2(nn.Module):
    """
    A Neural network with two convolution layers(with pooling) and three fully connected layers. 
    Note that nn.CrossEntropyLoss() has already realized softmax, so we do not need use softmax in forward()
    """
    def __init__(self):
        super(Net_2, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.loss = nn.CrossEntropyLoss() 

    def forward(self, z):
        """
        inputs:
            z: input for neural net (feature vector or a batch of features)
        outputs:
            output: given the current values of the neural net
               parameters, calculates the output of the
               neural net for the input of z
        """
        
        z = self.pool(F.relu(self.conv1(z)))
        z = self.pool(F.relu(self.conv2(z)))
        z = z.view(-1, 16 * 5 * 5)
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        output = self.fc3(z)

        return output

In [12]:
net2_1 = Net_2()
tic2_1 = time.time()
net2_1, loss2_1 = train(net2_1, trainloader_10, testloader_10, num_epochs = 20, step_size = 0.01, print_interval = 2500)
toc2_1 = time.time()

Epoch |  Interval  |     Loss     |  train  


/Users/always/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  1   |    2500    |    2.03770   |  0.400  
  1   |    5000    |    1.65014   |  0.300  
  2   |    2500    |    1.49681   |  0.700  
  2   |    5000    |    1.40636   |  0.600  
  3   |    2500    |    1.32654   |  0.400  
  3   |    5000    |    1.27903   |  0.700  
  4   |    2500    |    1.21499   |  0.300  
  4   |    5000    |    1.18294   |  0.800  
  5   |    2500    |    1.12450   |  0.700  
  5   |    5000    |    1.10778   |  0.900  
  6   |    2500    |    1.05372   |  0.600  
  6   |    5000    |    1.04667   |  0.600  
  7   |    2500    |    0.98750   |  0.700  
  7   |    5000    |    1.00019   |  0.600  
  8   |    2500    |    0.93440   |  0.800  
  8   |    5000    |    0.95764   |  0.600  
  9   |    2500    |    0.89158   |  0.700  
  9   |    5000    |    0.91384   |  0.700  
 10   |    2500    |    0.85105   |  0.700  
 10   |    5000    |    0.88086   |  0.700  
 11   |    2500    |    0.81807   |  0.500  
 11   |    5000    |    0.83864   |  0.900  
 12   |   

In [13]:
net2_2 = Net_2()
tic2_2 = time.time()
net2_2, loss2_2 = train(net2_2, trainloader_100, testloader_100, num_epochs = 20, step_size = 0.01, print_interval = 250)
toc2_2 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |     250    |    2.30249   |  0.060  
  1   |     500    |    2.30007   |  0.160  
  2   |     250    |    2.29550   |  0.180  
  2   |     500    |    2.28306   |  0.220  
  3   |     250    |    2.23501   |  0.190  
  3   |     500    |    2.10222   |  0.350  
  4   |     250    |    1.97464   |  0.330  
  4   |     500    |    1.90283   |  0.390  
  5   |     250    |    1.84478   |  0.330  
  5   |     500    |    1.78900   |  0.430  
  6   |     250    |    1.74902   |  0.430  
  6   |     500    |    1.70346   |  0.350  
  7   |     250    |    1.66426   |  0.400  
  7   |     500    |    1.64000   |  0.410  
  8   |     250    |    1.58915   |  0.350  
  8   |     500    |    1.56752   |  0.400  
  9   |     250    |    1.52820   |  0.470  
  9   |     500    |    1.51239   |  0.410  
 10   |     250    |    1.49619   |  0.440  
 10   |     500    |    1.46684   |  0.450  
 11   |     250    |    1.45109   |  0.530  
 11   |   

In [14]:
net2_3 = Net_2()
tic2_3 = time.time()
net2_3, loss2_3 = train(net2_3, trainloader_1000, testloader_1000, num_epochs = 20, step_size = 0.01, print_interval = 25)
toc2_3 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |      25    |    2.30386   |  0.106  
  1   |      50    |    2.30363   |  0.095  
  2   |      25    |    2.30347   |  0.100  
  2   |      50    |    2.30324   |  0.112  
  3   |      25    |    2.30270   |  0.103  
  3   |      50    |    2.30325   |  0.111  
  4   |      25    |    2.30258   |  0.117  
  4   |      50    |    2.30260   |  0.133  
  5   |      25    |    2.30256   |  0.101  
  5   |      50    |    2.30184   |  0.132  
  6   |      25    |    2.30195   |  0.130  
  6   |      50    |    2.30163   |  0.113  
  7   |      25    |    2.30134   |  0.125  
  7   |      50    |    2.30141   |  0.127  
  8   |      25    |    2.30122   |  0.136  
  8   |      50    |    2.30064   |  0.135  
  9   |      25    |    2.30057   |  0.134  
  9   |      50    |    2.30034   |  0.130  
 10   |      25    |    2.30005   |  0.148  
 10   |      50    |    2.29984   |  0.143  
 11   |      25    |    2.29960   |  0.134  
 11   |   

In [20]:
torch.save(net2_1, "net2_1")
torch.save(net2_2, "net2_2")
torch.save(net2_3, "net2_3")

In [29]:
# Your data
data = [
    loss2_1,
    loss2_2,
    loss2_3,
    [toc2_1-tic2_1, toc2_2-tic2_2, toc2_3-tic2_3]
]

# Open a file to write
with open('model2_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in data:
        writer.writerow(row)

## Net3, try activation function: tanh

In [7]:
class Net_3(nn.Module):
    """
    A Neural network with two convolution layers(with pooling) and three fully connected layers. 
    Note that nn.CrossEntropyLoss() has already realized softmax, so we do not need use softmax in forward()
    """
    def __init__(self):
        super(Net_3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.loss = nn.CrossEntropyLoss() 

    def forward(self, z):
        """
        inputs:
            z: input for neural net (feature vector or a batch of features)
        outputs:
            output: given the current values of the neural net
               parameters, calculates the output of the
               neural net for the input of z
        """
        
        z = self.pool(torch.tanh(self.conv1(z)))
        z = self.pool(torch.tanh(self.conv2(z)))
        z = z.view(-1, 16 * 5 * 5)
        z = torch.tanh(self.fc1(z))
        z = torch.tanh(self.fc2(z))
        output = self.fc3(z)

        return output

In [8]:
net3_1 = Net_3()
tic3_1 = time.time()
net3_1, loss3_1 = train(net3_1, trainloader_10, testloader_10, num_epochs = 20, step_size = 0.01, print_interval = 2500)
toc3_1 = time.time()

Epoch |  Interval  |     Loss     |  train  


/Users/always/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  1   |    2500    |    1.93698   |  0.400  
  1   |    5000    |    1.60076   |  0.400  
  2   |    2500    |    1.46201   |  0.500  
  2   |    5000    |    1.38442   |  0.500  
  3   |    2500    |    1.30619   |  0.700  
  3   |    5000    |    1.27533   |  0.600  
  4   |    2500    |    1.20449   |  0.600  
  4   |    5000    |    1.19869   |  0.800  
  5   |    2500    |    1.14471   |  0.800  
  5   |    5000    |    1.13505   |  0.400  
  6   |    2500    |    1.08139   |  0.700  
  6   |    5000    |    1.09445   |  0.700  
  7   |    2500    |    1.05096   |  0.500  
  7   |    5000    |    1.03904   |  0.800  
  8   |    2500    |    1.00984   |  0.700  
  8   |    5000    |    1.00484   |  0.500  
  9   |    2500    |    0.96415   |  0.600  
  9   |    5000    |    0.98411   |  0.500  
 10   |    2500    |    0.93046   |  0.600  
 10   |    5000    |    0.94858   |  0.700  
 11   |    2500    |    0.90403   |  0.700  
 11   |    5000    |    0.91818   |  0.700  
 12   |   

In [ ]:
net3_2 = Net_3()
tic3_2 = time.time()
net3_2, loss3_2 = train(net3_2, trainloader_100, testloader_100, num_epochs = 20, step_size = 0.01, print_interval = 250)
toc3_2 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |     250    |    2.28378   |  0.190  
  1   |     500    |    2.15489   |  0.240  
  2   |     250    |    2.02782   |  0.310  
  2   |     500    |    1.97650   |  0.230  
  3   |     250    |    1.94507   |  0.390  
  3   |     500    |    1.91636   |  0.310  
  4   |     250    |    1.88486   |  0.220  
  4   |     500    |    1.83196   |  0.420  
  5   |     250    |    1.77180   |  0.370  
  5   |     500    |    1.72327   |  0.360  
  6   |     250    |    1.67189   |  0.400  
  6   |     500    |    1.65970   |  0.410  
  7   |     250    |    1.62216   |  0.490  
  7   |     500    |    1.59822   |  0.410  
  8   |     250    |    1.57086   |  0.430  
  8   |     500    |    1.54188   |  0.340  
  9   |     250    |    1.51992   |  0.450  
  9   |     500    |    1.48490   |  0.420  
 10   |     250    |    1.47107   |  0.420  
 10   |     500    |    1.44267   |  0.450  
 11   |     250    |    1.42985   |  0.440  
 11   |   

In [16]:
net3_3 = Net_3()
tic3_3 = time.time()
net3_3, loss3_3 = train(net3_3, trainloader_1000, testloader_1000, num_epochs = 20, step_size = 0.01, print_interval = 25)
toc3_3 = time.time()

Epoch |  Interval  |     Loss     |  train  


KeyboardInterrupt: 

In [ ]:
torch.save(net3_1, "net3_1")
torch.save(net3_2, "net3_2")
torch.save(net3_3, "net3_3")

In [14]:
import csv
# Your data
data = [
    loss3_1,
    loss3_2,
    loss3_3,
    [toc3_1-tic3_1, toc3_2-tic3_2, toc3_3-tic3_3]
]

# Open a file to write
with open('model3_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in data:
        writer.writerow(row)

## Net4: More complex model

In [8]:
class Net_4(nn.Module):
    """
    A Neural network with two convolution layers(with pooling) and three fully connected layers. 
    Note that nn.CrossEntropyLoss() has already realized softmax, so we do not need use softmax in forward()
    """
    def __init__(self):
        super(Net_4, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 
        self.bn1 = nn.BatchNorm2d(32)               
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)               
        self.pool = nn.MaxPool2d(2, 2)              
        self.fc1 = nn.Linear(64 * 8 * 8, 512)       
        self.fc2 = nn.Linear(512, 10)   

    def forward(self, z):
        """
        inputs:
            z: input for neural net (feature vector or a batch of features)
        outputs:
            output: given the current values of the neural net
               parameters, calculates the output of the
               neural net for the input of z
        """
        
        z = self.pool(F.relu(self.bn1(self.conv1(z)))) 
        z = self.pool(F.relu(self.bn2(self.conv2(z)))) 
        z = z.view(-1, 64 * 8 * 8)                     
        z = F.relu(self.fc1(z))                        
        output = self.fc2(z)

        return output

In [17]:
net4_1 = Net_4()
tic4_1 = time.time()
net4_1, loss4_1 = train(net4_1, trainloader_10, testloader_10, num_epochs = 20, step_size = 0.01, print_interval = 2500)
toc4_1 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |    2500    |    1.42804   |  0.500  
  1   |    5000    |    1.10063   |  0.800  
  2   |    2500    |    0.92786   |  0.600  
  2   |    5000    |    0.87517   |  0.700  
  3   |    2500    |    0.74380   |  0.800  
  3   |    5000    |    0.73103   |  0.800  
  4   |    2500    |    0.60822   |  0.500  
  4   |    5000    |    0.60561   |  1.000  
  5   |    2500    |    0.48255   |  0.800  
  5   |    5000    |    0.49623   |  0.800  
  6   |    2500    |    0.37111   |  0.800  
  6   |    5000    |    0.40076   |  1.000  
  7   |    2500    |    0.27974   |  0.800  
  7   |    5000    |    0.30367   |  0.900  
  8   |    2500    |    0.20244   |  0.800  
  8   |    5000    |    0.22310   |  0.900  
  9   |    2500    |    0.14411   |  1.000  
  9   |    5000    |    0.15782   |  1.000  
 10   |    2500    |    0.09100   |  0.900  
 10   |    5000    |    0.11361   |  0.900  
 11   |    2500    |    0.07127   |  1.000  
 11   |   

In [ ]:
net4_2 = Net_4()
tic4_2 = time.time()
net4_2, loss4_2 = train(net4_2, trainloader_100, testloader_100, num_epochs = 20, step_size = 0.01, print_interval = 250)
toc4_2 = time.time()

Epoch |  Interval  |     Loss     |  train  
  1   |     250    |    1.67561   |  0.520  
  1   |     500    |    1.33816   |  0.530  
  2   |     250    |    1.19601   |  0.640  
  2   |     500    |    1.12218   |  0.600  
  3   |     250    |    1.04284   |  0.680  
  3   |     500    |    0.99458   |  0.690  
  4   |     250    |    0.93735   |  0.680  
  4   |     500    |    0.91977   |  0.650  
  5   |     250    |    0.87926   |  0.660  
  5   |     500    |    0.84783   |  0.700  
  6   |     250    |    0.81390   |  0.730  
  6   |     500    |    0.80295   |  0.800  
  7   |     250    |    0.75696   |  0.760  
  7   |     500    |    0.75822   |  0.720  
  8   |     250    |    0.71228   |  0.730  
  8   |     500    |    0.71598   |  0.760  
  9   |     250    |    0.66870   |  0.770  
  9   |     500    |    0.67478   |  0.780  
 10   |     250    |    0.62344   |  0.830  
 10   |     500    |    0.62604   |  0.830  
 11   |     250    |    0.59522   |  0.760  
 11   |   

In [10]:
net4_3 = Net_4()
tic4_3 = time.time()
net4_3, loss4_3 = train(net4_3, trainloader_1000, testloader_1000, num_epochs = 20, step_size = 0.01, print_interval = 25)
toc4_3 = time.time()

Epoch |  Interval  |     Loss     |  train  


/Users/always/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  1   |      25    |    2.14072   |  0.344  
  1   |      50    |    1.87284   |  0.376  
  2   |      25    |    1.72549   |  0.407  
  2   |      50    |    1.64221   |  0.443  
  3   |      25    |    1.58083   |  0.472  
  3   |      50    |    1.52452   |  0.493  
  4   |      25    |    1.48896   |  0.489  
  4   |      50    |    1.44335   |  0.524  
  5   |      25    |    1.40783   |  0.550  
  5   |      50    |    1.38904   |  0.497  
  6   |      25    |    1.35397   |  0.540  
  6   |      50    |    1.32616   |  0.562  
  7   |      25    |    1.29960   |  0.545  
  7   |      50    |    1.27939   |  0.504  
  8   |      25    |    1.26188   |  0.552  
  8   |      50    |    1.23421   |  0.578  
  9   |      25    |    1.22112   |  0.588  
  9   |      50    |    1.20867   |  0.567  
 10   |      25    |    1.19434   |  0.599  
 10   |      50    |    1.17387   |  0.612  
 11   |      25    |    1.16160   |  0.600  
 11   |      50    |    1.15815   |  0.592  
 12   |   

In [ ]:
torch.save(net4_1, "net4_1")
torch.save(net4_2, "net4_2")

In [ ]:
# Your data
data = [
    loss4_1,
    loss4_2,
    loss4_3,
    [toc4_1-tic4_1, toc4_2-tic4_2, toc4_3-tic4_3]
]

# Open a file to write
with open('model4_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in data:
        writer.writerow(row)

In [13]:
torch.save(net4_3, "net4_3")

In [14]:
import csv
# Your data
data = [
    loss4_3,
    [toc4_3-tic4_3]
]

# Open a file to write
with open('model4_3_loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in data:
        writer.writerow(row)